In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
import pandas as pd


In [2]:
df = pd.read_csv("ml-latest-small/ratings.csv")
df_small = df.head(900)


In [9]:

matrix = df.pivot(index='userId', columns='movieId', values='rating')

matrix_array = matrix.to_numpy()

matrix_array[matrix_array == 0] = np.nan



In [10]:

def factorize(A, k):
    """Factorize the matrix A into W and H"""
    # Randomly initialize W and H
    W = tf.Variable(tf.random.normal((A.shape[0], k), dtype=tf.float32))
    H = tf.Variable(tf.random.normal((k, A.shape[1]), dtype=tf.float32))
    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    # Train the model
    for i in range(1000):
        with tf.GradientTape() as tape:
            # Compute the loss
            diff_matrix = tf.matmul(W, H) - A
            mask = ~tf.math.is_nan(A)
            masked_diff = tf.boolean_mask(diff_matrix, mask)
            loss = tf.norm(masked_diff)
        
        # Compute gradients
        gradients = tape.gradient(loss, [W, H])
        
        # Update the parameters
        optimizer.apply_gradients(zip(gradients, [W, H]))
        
    return W, H, loss


In [11]:

W, H, loss = factorize(matrix_array, 5)


In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error

def calculate_rmse(predicted_ratings, observed_ratings):
    # Remove NaN values from the observed ratings and the corresponding predicted ratings
    observed_ratings_non_nan = observed_ratings[~np.isnan(observed_ratings)]
    predicted_ratings_non_nan = predicted_ratings[~np.isnan(observed_ratings)]
    
    # Calculate the RMSE
    rmse = np.sqrt(mean_squared_error(observed_ratings_non_nan, predicted_ratings_non_nan))
    return rmse


In [15]:
def predict_movie_ratings(user, df, W, H):
    idx = df.index.get_loc(user)
    user_ratings = df.iloc[idx]
    user_ratings = user_ratings.dropna()
    user_ratings = user_ratings.to_frame().T
    user_ratings = user_ratings.reindex(columns=df.columns, fill_value=float('nan'))
    user_ratings = user_ratings.to_numpy()

    predicted_ratings = tf.matmul(W, H).numpy()
    predicted_ratings_user = predicted_ratings[idx].flatten()
    
    # Calculate RMSE
    rmse = calculate_rmse(predicted_ratings_user, user_ratings.flatten())
    
    return pd.DataFrame({"Observed": user_ratings.flatten(), "Predicted": predicted_ratings_user}), rmse


In [16]:
predict_movie_ratings(1, matrix, W, H)


(      Observed  Predicted
 0          4.0   4.621564
 1          NaN   4.229770
 2          4.0   4.254331
 3          NaN   2.156176
 4          NaN   3.808494
 ...        ...        ...
 9719       NaN   2.382705
 9720       NaN   0.576959
 9721       NaN   2.571400
 9722       NaN   2.385490
 9723       NaN   4.527680
 
 [9724 rows x 2 columns],
 0.6756355680839877)